In [38]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"

In [27]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [28]:
model.invoke("Where is Berlin?")

AIMessage(content='Berlin is the capital city of Germany, located in the northeastern part of the country.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6895b6a1-3c44-48f2-80bd-53fd79b24a6e-0', usage_metadata={'input_tokens': 11, 'output_tokens': 17, 'total_tokens': 28})

In [29]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Where is Berlin?")

'Berlin is the capital city of Germany and is located in the northeastern part of the country.'

In [30]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Maksym is a software engineer", question="What is Maksym's job?")

'Human: \nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Maksym is a software engineer\n\nQuestion: What is Maksym\'s job?\n'

In [31]:
chain = prompt | model | parser
chain.invoke({
    "context": "Maksym is a software engineer",
    "question": "What is Maksym's job?"
})

"Maksym's job is a software engineer."

In [32]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

In [33]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Alina works at Schulflix and meinUnterricht",
        "question": "How many jobs does Alina have?",
        "language": "German",
    }
)

'Alina hat zwei Jobs.'

In [40]:
import tempfile
import whisper
from pytube import YouTube


# Let's do this only if we haven't created the transcription file yet.
if not os.path.exists("transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()

    # Let's load the base model. This is not the most accurate
    # model but it's fast.
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("transcription.txt", "w") as file:
            file.write(transcription)

In [42]:
with open("transcription.txt") as file:
    transcription = file.read()

transcription[:100]

'Ist die Tomate ein Obst oder Gemüse?\nIm Supermarkt finden wir die Tomate in der Gemüseabteilung. Doc'

In [44]:
try:
    chain.invoke({
        "context": transcription,
        "question": "Tomate Obst oder Gemüse?"
    })
except Exception as e:
    print(e)

In [45]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("transcription.txt")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'transcription.txt'}, page_content='Ist die Tomate ein Obst oder Gemüse?\nIm Supermarkt finden wir die Tomate in der Gemüseabteilung. Doch liegt sie da richtig? Schauen wir uns das mal an.\nDu bist auf der Suche nach einer passenden Ausbildung? Dann komm aufs studyfl.de und findet 10.000 offene Ausbildungsstellen.\nTomate ganz klar Gemüse. Das würden wahrscheinlich die meisten Menschen antworten, wenn sie auf der Straße gefragt werden. Doch ist die Antwort wirklich so einfach? So eindeutig wie die Einteilung der Tomate auf den ersten Blick erscheint, ist sie nämlich leider nicht. Denn eigentlich ist die Tomate beides Obst und Gemüse. Doch wieso ist das so?\nTatsächlich gibt es mehrere Kriterien, nach denen Lebensmittel zum Obst oder zum Gemüse zugeordnet werden. Gärtner teilen Pflanzen immer nach den äußeren Merkmalen, also nach den botanischen Kriterien, ein. In der Küche sind die botanischen Kriterien jedoch weniger von Interesse und es zählt mehr der Ge

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(text_documents)[:5]

[Document(metadata={'source': 'transcription.txt'}, page_content='Ist die Tomate ein Obst oder Gemüse?'),
 Document(metadata={'source': 'transcription.txt'}, page_content='Im Supermarkt finden wir die Tomate in der Gemüseabteilung. Doch liegt sie da richtig? Schauen wir'),
 Document(metadata={'source': 'transcription.txt'}, page_content='Schauen wir uns das mal an.'),
 Document(metadata={'source': 'transcription.txt'}, page_content='Du bist auf der Suche nach einer passenden Ausbildung? Dann komm aufs studyfl.de und findet 10.000'),
 Document(metadata={'source': 'transcription.txt'}, page_content='und findet 10.000 offene Ausbildungsstellen.')]

In [47]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

In [48]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("Where does Alina work?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[0.012111732736229897, -0.018987061455845833, -0.021503256633877754, -0.024223465472459793, -0.019150273874402046, 0.015668407082557678, -0.05353372171521187, 0.010948843322694302, -0.005137795582413673, -0.01100324746221304]


In [49]:
sentence1 = embeddings.embed_query("Alina works at Schulflix and meinUnterricht")
sentence2 = embeddings.embed_query("Maksym is a software engineer")

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.8795154849803621, 0.7823709884681839)

In [51]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Laura in lives Berlin",
        "Maksym lives in Bavaria",
        "Christian lives in Berlin",
        "Alina lives in Kölln",
        "Laura is a Marketing expert",
        "Maksym knows four languages",
    ],
    embedding=embeddings,
)

In [53]:
vectorstore1.similarity_search_with_score(query="What does Maksym know?", k=3)

[(Document(page_content='Maksym knows four languages'), 0.9127768608432575),
 (Document(page_content='Maksym lives in Bavaria'), 0.883141742360861),
 (Document(page_content='Alina lives in Kölln'), 0.7666814374372494)]

In [54]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Where does Laura live?")

[Document(page_content='Laura in lives Berlin'),
 Document(page_content='Laura is a Marketing expert'),
 Document(page_content='Alina lives in Kölln'),
 Document(page_content='Christian lives in Berlin')]

In [55]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("What language does Maksym know?")

{'context': [Document(page_content='Maksym knows four languages'),
  Document(page_content='Maksym lives in Bavaria'),
  Document(page_content='Alina lives in Kölln'),
  Document(page_content='Christian lives in Berlin')],
 'question': 'What language does Maksym know?'}

In [56]:
chain = setup | prompt | model | parser
chain.invoke("What's Laura's job?")

'Laura is a Marketing expert.'

In [57]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [58]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("Tomate Obst oder Gemüse?")

'Die Tomate gehört sowohl zu Obst als auch zu Gemüse.'

In [61]:
from langchain_pinecone import PineconeVectorStore

index_name = "mu-rag-index"

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

In [62]:
pinecone.similarity_search("Was ist die botanische Klassifizierung?")[:3]

[Document(metadata={'source': 'transcription.txt'}, page_content='Tatsächlich gibt es mehrere Kriterien, nach denen Lebensmittel zum Obst oder zum Gemüse zugeordnet werden. Gärtner teilen Pflanzen immer nach den äußeren Merkmalen, also nach den botanischen Kriterien, ein. In der Küche sind die botanischen Kriterien jedoch weniger von Interesse und es zählt mehr der Geschmack und die Art der Zubereitung. Das sind die kulinarischen Kriterien. Gärtner stellen sich bei der Einteilung in Obst und Gemüse vor allem zwei Fragen: Wie alt kann die Pflanze werden und welche Teile davon ist man? Ist die Pflanze älter als zwei Jahre alt, also eine mehrjährige Pflanze, dann handelt es sich wahrscheinlich um eine Obstpflanze. Das können Bäume oder Sträucher sein, die mehrere Jahre oder Jahrzehnte leben. Jedes Jahr bekommen die Pflanzen Blüten, aus denen sich nach der Bestäubung Früchte bilden. Die Früchte essen wir als Obst, dazu gehören zum Beispiel Äpfel, Pflaumen oder Orangen. Wenn die Pflanze hin

In [63]:
chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

chain.invoke("Ist die Tomate ein Gemüse?")

'Die Tomate ist sowohl Obst als auch Gemüse.'